In [2]:
# docker daemon manages containers
# The daemon exposes a REST API
# this API is then used by docker CLI, docker Python/Go SDK, etc

import time
import threading

def spawn(func):
    thread = threading.Thread(target=func)
    thread.start()
    return thread

import docker
client = docker.from_env()
client2 = docker.DockerClient(base_url='unix://var/run/docker.sock')

In [3]:
print(client.ping())
print(client.containers.list())
client.containers.run("bfirsh/reticulate-splines", detach=True)
print(client.containers.list())

True
[<Container: c16b0ff4a67b>]


[<Container: 3c44017d1718>, <Container: c16b0ff4a67b>]


In [7]:
def kill_all():
    for container in client.containers.list(all=True):
        container.kill()
kill_all()
time.sleep(0.1)
print(client.containers.list())

[]


In [5]:

# besides actively query info, 
#   we can also track changes with websocket

@spawn
def watch():
    global events
    events = client.events(decode=True)
    for event in events:
        print(event)

cont = client.containers.run("bfirsh/reticulate-splines", detach=True)
time.sleep(0.1)
cont.kill()

# stop the watch thread
events.close()

{'status': 'create', 'id': '8f479d77d675e029a63616f09ce91a9814def1be7a42edf1fd464b4705984ffa', 'from': 'bfirsh/reticulate-splines', 'Type': 'container', 'Action': 'create', 'Actor': {'ID': '8f479d77d675e029a63616f09ce91a9814def1be7a42edf1fd464b4705984ffa', 'Attributes': {'image': 'bfirsh/reticulate-splines', 'name': 'zen_thompson'}}, 'scope': 'local', 'time': 1705568164, 'timeNano': 1705568164603740167}
{'Type': 'network', 'Action': 'connect', 'Actor': {'ID': '75d500952ece1d1913a6bb68e2134cae056c1bfac5f4b7cd5516619e70aad0e0', 'Attributes': {'container': '8f479d77d675e029a63616f09ce91a9814def1be7a42edf1fd464b4705984ffa', 'name': 'bridge', 'type': 'bridge'}}, 'scope': 'local', 'time': 1705568164, 'timeNano': 1705568164653041943}


{'status': 'start', 'id': '8f479d77d675e029a63616f09ce91a9814def1be7a42edf1fd464b4705984ffa', 'from': 'bfirsh/reticulate-splines', 'Type': 'container', 'Action': 'start', 'Actor': {'ID': '8f479d77d675e029a63616f09ce91a9814def1be7a42edf1fd464b4705984ffa', 'Attributes': {'image': 'bfirsh/reticulate-splines', 'name': 'zen_thompson'}}, 'scope': 'local', 'time': 1705568164, 'timeNano': 1705568164863664423}
{'status': 'kill', 'id': '8f479d77d675e029a63616f09ce91a9814def1be7a42edf1fd464b4705984ffa', 'from': 'bfirsh/reticulate-splines', 'Type': 'container', 'Action': 'kill', 'Actor': {'ID': '8f479d77d675e029a63616f09ce91a9814def1be7a42edf1fd464b4705984ffa', 'Attributes': {'image': 'bfirsh/reticulate-splines', 'name': 'zen_thompson', 'signal': '9'}}, 'scope': 'local', 'time': 1705568164, 'timeNano': 1705568164982225888}
{'Type': 'network', 'Action': 'disconnect', 'Actor': {'ID': '75d500952ece1d1913a6bb68e2134cae056c1bfac5f4b7cd5516619e70aad0e0', 'Attributes': {'container': '8f479d77d675e029a636

In [8]:
# docker api
import time
import docker
daemon = docker.from_env()

for container in daemon.containers.list(all=True):
    container.remove(force=True)
time.sleep(0.1)

container = daemon.containers.run(
    "my-python-env",
    detach=True,
    ports={
        22: 0,
        8888: 0,
    },
    name="my-python-env",
)
time.sleep(0.5)

# container simply stores the id and attrs
# the real data query happens when we call the methods
print(container.logs().decode())
# refresh attrs with id
container.reload()
display(container.attrs['NetworkSettings']['Ports'])
print(daemon.containers.list(all=True))

# if we kill the container, reload will fail with 404 no such container
container.remove(force=True)
time.sleep(0.1)
try:
    container.reload()
except docker.errors.NotFound:
    print("container not found")# docker api
import time
import docker
daemon = docker.from_env()

for container in daemon.containers.list(all=True):
    container.remove(force=True)
time.sleep(0.1)

container = daemon.containers.run(
    "my-python-env",
    detach=True,
    ports={
        22: 0,
        8888: 0,
    },
    name="my-python-env",
)
time.sleep(0.5)

# container simply stores the id and attrs
# the real data query happens when we call the methods
print(container.logs().decode())
# refresh attrs with id
container.reload()
display(container.attrs['NetworkSettings']['Ports'])
display(container.attrs)

print(daemon.containers.list(all=True))

# if we kill the container, reload will fail with 404 no such container
container.remove(force=True)
time.sleep(0.1)
try:
    container.reload()
except docker.errors.NotFound:
    print("container not found")

{'22/tcp': [{'HostIp': '0.0.0.0', 'HostPort': '32817'},
  {'HostIp': '::', 'HostPort': '32817'}],
 '8888/tcp': [{'HostIp': '0.0.0.0', 'HostPort': '32816'},
  {'HostIp': '::', 'HostPort': '32816'}]}

[<Container: 2d854a0072ad>]
container not found



{'22/tcp': [{'HostIp': '0.0.0.0', 'HostPort': '32819'},
  {'HostIp': '::', 'HostPort': '32819'}],
 '8888/tcp': [{'HostIp': '0.0.0.0', 'HostPort': '32818'},
  {'HostIp': '::', 'HostPort': '32818'}]}

{'Id': 'b34f53c609ae4dc61f8ef6924a220b1cf2cf03778538c5b26cc49d115789b4d4',
 'Created': '2024-01-18T09:06:53.698180909Z',
 'Path': '/bin/sh',
 'Args': ['-c',
  '/usr/sbin/sshd -D  && jupyter notebook --ip=0.0.0.0 --port=8888 --allow-root'],
 'State': {'Status': 'running',
  'Running': True,
  'Paused': False,
  'Restarting': False,
  'OOMKilled': False,
  'Dead': False,
  'Pid': 365651,
  'ExitCode': 0,
  'Error': '',
  'StartedAt': '2024-01-18T09:06:54.053057267Z',
  'FinishedAt': '0001-01-01T00:00:00Z'},
 'Image': 'sha256:be74893fcdb8509f7b252fd8b628f58c5d1a07f24c59e927ddc348039f532292',
 'ResolvConfPath': '/var/lib/docker/containers/b34f53c609ae4dc61f8ef6924a220b1cf2cf03778538c5b26cc49d115789b4d4/resolv.conf',
 'HostnamePath': '/var/lib/docker/containers/b34f53c609ae4dc61f8ef6924a220b1cf2cf03778538c5b26cc49d115789b4d4/hostname',
 'HostsPath': '/var/lib/docker/containers/b34f53c609ae4dc61f8ef6924a220b1cf2cf03778538c5b26cc49d115789b4d4/hosts',
 'LogPath': '/var/lib/docker/containers/b3

[<Container: b34f53c609ae>]
container not found
